In [3]:
# %%
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV

import matplotlib.pyplot as plt
#import seaborn as sns
import pickle



# %%
YEARS = [2018, 2019, 2020, 2021, 2022, 2023,2024]

# %%
data_all = pd.DataFrame()

def calculate_seconds(row):
    if row['qtr'] != 5:
        return 3600 - row['game_seconds_remaining']
    else:
        return 600 - row['game_seconds_remaining'] + 3600


def get_quarter_value(dataf):
    if 'END QUARTER' in dataf['desc']:
        return dataf['level_0']
    else:
        return None

for i in YEARS:  
    i_data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(i) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data_all = pd.concat([data_all,i_data])

ppr = 1

data = data_all.loc[data_all.season_type=='REG']
#data = data_all.loc[(data_all.play_type.isin(['no_play','pass','run'])) & (data_all.epa.isna()==False)]
#data.loc[data['pass']==1, 'play_type'] = 'pass'
#data.loc[data.rush==1, 'play_type'] = 'run'
data.reset_index(drop=True, inplace=True)
data['turnover'] = data['interception'] + data['fumble_lost']
data = data.dropna(subset=['posteam'])
data['inside_10'] = (data['yardline_100'] < 10).astype(int)
data['20+_play'] = (data['yards_gained'] > 19).astype(int)
data['short_pass'] = (data['air_yards'] < 10).astype(int)
data['medium_pass'] = ((data['air_yards'] > 9)&(data['air_yards']<20)).astype(int)
data['deep_pass'] = (data['air_yards'] > 19).astype(int)
data['end_zone_target'] = (data['yardline_100'] - data['air_yards']) <= 0
data['fantasy_points'] = (
    data['complete_pass'] * ppr +          # 1 point per completion
    data['touchdown'] * 6 +           # 6 points per touchdown
    data['yards_gained'] * 0.1        # 0.1 points per yard gained
)
data['distance_to_EZ_after_target'] = data['yardline_100'] - data['air_yards']


C:\Users\src15\AppData\Local\Temp\ipykernel_21008\2018476112.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_21008\2018476112.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_21008\2018476112.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inser

In [4]:
def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [5]:
    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]


    # derive implied team total from betting market data
    data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
    data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)

    # Use list comprehension with zip for more efficient row-wise operations
    data['implied_posteam_total'] = [
    total_finder(has_ball, home_number, away_number)
        for has_ball, home_number, away_number in zip(data['posteam_type'], data['home_implied_total'], data['away_implied_total'])
    ]

C:\Users\src15\AppData\Local\Temp\ipykernel_21008\2215997381.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_21008\2215997381.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_21008\2215997381.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the r

In [6]:
    
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    #throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


In [7]:
def load_models():
    with open('Models/yardage_model.pkl', 'rb') as file:
        yardage_model = pickle.load(file)
    
    with open('Models/touchdown_model.pkl', 'rb') as file:
        touchdown_model = pickle.load(file)

    return yardage_model, touchdown_model

In [8]:
yardage_model, touchdown_model = load_models()

In [9]:
df['season'].value_counts()

season
2021    18055
2023    17483
2020    17307
2022    17305
2018    17172
2019    17142
2024    17013
Name: count, dtype: int64

In [10]:
new_predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]

test_df = df[new_predictors]

test_df.head()

,air_yards,yardline_100,ydstogo,down,pass_location,qb_hit,end_zone_target,distance_to_EZ_after_target
2,8.0,80.0,15,1.0,right,0.0,False,72.0
5,4.0,39.0,10,1.0,right,0.0,False,35.0
6,-3.0,39.0,10,2.0,left,0.0,False,42.0
7,24.0,39.0,10,3.0,left,0.0,False,15.0
10,1.0,1.0,1,3.0,right,0.0,True,0.0


In [11]:
test_df = pd.get_dummies(test_df, columns=['pass_location'], drop_first=True)


In [12]:
# Add predictions to the new dataset (optional)
df['xYards'] = yardage_model.predict(test_df)
df['xTDs'] = touchdown_model.predict_proba(test_df)[:, 1]
#df['xFPs'] = best_model.predict(test_df)
df['compositeXFP'] = df['cp'] * ppr + df['xTDs'] * 6 + df['xYards'] * 0.1

C:\Users\src15\AppData\Local\Temp\ipykernel_21008\203326306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xYards'] = yardage_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_21008\203326306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xTDs'] = touchdown_model.predict_proba(test_df)[:, 1]
C:\Users\src15\AppData\Local\Temp\ipykernel_21008\203326306.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [13]:
receivers = df.groupby(['receiver_player_id','posteam','game_id']).agg({'receiver_player_name':'max','pass':'sum','week':'max','season':'max','xTDs':'sum','pass_touchdown':'sum','xYards':'sum','yards_gained':'sum','cp':'sum','complete_pass':'sum','fantasy_points':'sum','compositeXFP':'sum','implied_posteam_total':'max'}).sort_values('compositeXFP',ascending=False)

# Trailing 3 Weeks

In [14]:
df_1 = receivers.sort_values('xYards',ascending=False)

In [15]:
df = df_1.reset_index()

In [16]:

# --- Sort correctly ---
df = df.sort_values(by=["receiver_player_id", "season", "week"]).reset_index(drop=True)

# --- Metrics to use for trailing windows ---
metrics = ["pass", "xTDs", "pass_touchdown", "xYards", "yards_gained", 
           "cp", "complete_pass", "fantasy_points", "compositeXFP"]

grouped = df.groupby(["receiver_player_id", "season"])

# --- Compute rolling 3-week totals ---
rolling_sum = grouped[metrics].rolling(window=2, min_periods=2).sum().reset_index(level=[0,1], drop=True)
# Shift so weeks 1–3 line up with week 4
rolling_sum = rolling_sum.groupby([df["receiver_player_id"], df["season"]]).shift(1)

# --- Compute rolling 3-week averages ---
#rolling_avg = grouped[metrics].rolling(window=3, min_periods=3).mean().reset_index(level=[0,1], drop=True)
#rolling_avg = rolling_avg.groupby([df["receiver_player_id"], df["season"]]).shift(1)

# --- Attach both sets of features ---
for col in metrics:
    df[f"{col}_last3_sum"] = rolling_sum[col]
    #df[f"{col}_last3_avg"] = rolling_avg[col]

# --- Target is the *current* week’s fantasy points ---
df["fantasy_points_target"] = df["fantasy_points"]

# --- Keep only rows where we have full trailing history ---
df_model = df.dropna(subset=[f"{col}_last3_sum" for col in metrics] + 
                     ["fantasy_points_target"])

print(df_model.head(10))


   receiver_player_id posteam          game_id receiver_player_name  pass  \
5          00-0021547     LAC   2018_03_LAC_LA              A.Gates     5   
6          00-0021547     LAC   2018_04_SF_LAC              A.Gates     4   
7          00-0021547     LAC  2018_05_OAK_LAC              A.Gates     2   
8          00-0021547     LAC  2018_06_LAC_CLE              A.Gates     2   
9          00-0021547     LAC  2018_07_TEN_LAC              A.Gates     3   
10         00-0021547     LAC  2018_10_LAC_OAK              A.Gates     2   
11         00-0021547     LAC  2018_11_DEN_LAC              A.Gates     7   
12         00-0021547     LAC  2018_12_ARI_LAC              A.Gates     1   
13         00-0021547     LAC  2018_13_LAC_PIT              A.Gates     1   
14         00-0021547     LAC  2018_14_CIN_LAC              A.Gates     2   

    week  season      xTDs  pass_touchdown     xYards  ...  pass_last3_sum  \
5      3    2018  0.337634             0.0  34.707394  ...             4.0

In [17]:
df_model[df_model.columns[7:]].corr()['fantasy_points'].sort_values(ascending=False)

fantasy_points              1.000000
fantasy_points_target       1.000000
yards_gained                0.910264
complete_pass               0.855410
compositeXFP                0.806602
xYards                      0.757053
cp                          0.747438
pass_touchdown              0.711676
xTDs                        0.572098
compositeXFP_last3_sum      0.484697
xYards_last3_sum            0.483445
pass_last3_sum              0.480457
cp_last3_sum                0.453285
yards_gained_last3_sum      0.450956
fantasy_points_last3_sum    0.447708
complete_pass_last3_sum     0.447632
xTDs_last3_sum              0.327328
pass_touchdown_last3_sum    0.231748
implied_posteam_total       0.106299
Name: fantasy_points, dtype: float64

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# --- Define features and target ---
feature_cols = [col for col in df_model.columns if col.endswith("_last3_sum") or col.endswith("_last3_avg") or col.endswith("team_total")]
X = df_model[feature_cols]
y = df_model["fantasy_points_target"]

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Random Forest Regressor ---
rf = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# --- Predictions ---
y_pred = rf.predict(X_test)

# --- Evaluation ---
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse:.2f}, R²: {r2:.3f}")

# --- Feature Importance ---
importances = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=False)
print("\nTop features:")
print(importances.head(20))

# --- Example: predict next game fantasy points for last row ---
example_features = X.iloc[[-1]]
example_pred = rf.predict(example_features)
print(f"\nPredicted fantasy points for last row: {example_pred[0]:.2f}")


RMSE: 6.19, R²: 0.235

Top features:
xYards_last3_sum            0.260456
compositeXFP_last3_sum      0.184151
xTDs_last3_sum              0.106912
cp_last3_sum                0.104553
implied_posteam_total       0.097414
fantasy_points_last3_sum    0.093790
yards_gained_last3_sum      0.091977
complete_pass_last3_sum     0.033202
pass_last3_sum              0.019221
pass_touchdown_last3_sum    0.008325
dtype: float64

Predicted fantasy points for last row: 2.24


In [ ]:

from sklearn.linear_model import LinearRegression

# --- Use only compositeXFP_last3_sum as feature ---
X = df_model[["compositeXFP_last3_sum"]]
y = df_model["fantasy_points_target"]

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Simple Linear Regression ---
lr = LinearRegression()
lr.fit(X_train, y_train)

# --- Predictions ---
y_pred = lr.predict(X_test)

# --- Evaluation ---
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}, R²: {r2:.3f}")
print(f"Model equation: FantasyPoints ≈ {lr.intercept_:.2f} + {lr.coef_[0]:.2f} * compositeXFP_last3_sum")

# --- Example prediction for the last row ---
example_features = X.iloc[[-1]]
example_pred = lr.predict(example_features)
print(f"\nPredicted fantasy points for last row: {example_pred[0]:.2f}")


RMSE: 6.14, R²: 0.247
Model equation: FantasyPoints ≈ 2.41 + 0.34 * compositeXFP_last3_sum

Predicted fantasy points for last row: 3.89


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# --- Features + target ---
X = df_model[["compositeXFP_last3_sum", "pass_last3_sum"]]#, "implied_posteam_total"]]
y = df_model["fantasy_points_target"]

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Linear Regression ---
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_pred))
lr_r2 = r2_score(y_test, lr_pred)

# --- Random Forest ---
rf = RandomForestRegressor(n_estimators=500, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_r2 = r2_score(y_test, rf_pred)

# --- Results ---
print("Linear Regression:")
print(f"  RMSE: {lr_rmse:.2f}, R²: {lr_r2:.3f}")
print("  Equation:")
print(f"    FantasyPoints ≈ {lr.intercept_:.2f} "
      f"+ {lr.coef_[0]:.2f} * compositeXFP_last3_sum "
      f"+ {lr.coef_[1]:.2f} * pass_last3_sum ")
      #f"+ {lr.coef_[2]:.2f} * implied_posteam_total")

print("\nRandom Forest:")
print(f"  RMSE: {rf_rmse:.2f}, R²: {rf_r2:.3f}")
print("  Feature importances:")
for feat, imp in zip(X.columns, rf.feature_importances_):
    print(f"    {feat}: {imp:.3f}")

# --- Example prediction for most recent row ---
example_features = X.iloc[[-1]]
print("\nExample prediction (most recent row):")
print(f"Linear Regression: {lr.predict(example_features)[0]:.2f}")
print(f"Random Forest:     {rf.predict(example_features)[0]:.2f}")


Linear Regression:
  RMSE: 6.13, R²: 0.248
  Equation:
    FantasyPoints ≈ 2.31 + 0.25 * compositeXFP_last3_sum + 0.16 * pass_last3_sum 

Random Forest:
  RMSE: 7.07, R²: 0.001
  Feature importances:
    compositeXFP_last3_sum: 0.913
    pass_last3_sum: 0.087

Example prediction (most recent row):
Linear Regression: 3.89
Random Forest:     1.91


In [21]:
with open('in_szn_model_wr_model.pkl', 'wb') as file:
    pickle.dump(lr, file)

# Season-over-season Comparison

In [11]:
receivers = df.groupby(['receiver_player_id','posteam','season']).agg({'receiver_player_name':'max','pass':'sum','xTDs':'sum','pass_touchdown':'sum','xYards':'sum','yards_gained':'sum','cp':'sum','complete_pass':'sum','fantasy_points':'sum','compositeXFP':'sum'}).sort_values('compositeXFP',ascending=False)

In [12]:
receivers

,,,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,complete_pass,fantasy_points,compositeXFP
receiver_player_id,posteam,season,,,,,,,,,,
00-0033908,LA,2021,C.Kupp,191,11.596006,16.0,1438.502686,1947.0,129.081419,145.0,441.7,342.507733
00-0036358,DAL,2023,C.Lamb,181,12.352115,12.0,1393.984985,1749.0,117.402901,135.0,381.9,330.914094
00-0031381,LV,2022,D.Adams,180,10.494546,14.0,1526.758789,1516.0,112.967685,100.0,341.6,328.610843
00-0036322,MIN,2022,J.Jefferson,184,10.750579,8.0,1458.707886,1819.0,116.943704,128.0,357.9,327.317971
00-0036900,CIN,2024,J.Chase,175,12.978581,17.0,1284.327271,1708.0,112.442590,127.0,405.8,318.746810
...,...,...,...,...,...,...,...,...,...,...,...,...
00-0035289,JAX,2020,G.Minshew II,1,0.000601,0.0,0.969162,0.0,0.596584,1.0,1.0,0.697108
00-0019596,NE,2018,T.Brady,1,0.002768,0.0,3.223743,6.0,0.329838,1.0,1.6,0.668823
00-0029263,SEA,2018,R.Wilson,1,0.002653,0.0,0.449581,-11.0,0.570121,1.0,-0.1,0.630997


In [13]:
receivers_qual = receivers[receivers['pass']>=20]

In [14]:
receivers

,,,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,complete_pass,fantasy_points,compositeXFP
receiver_player_id,posteam,season,,,,,,,,,,
00-0033908,LA,2021,C.Kupp,191,11.596006,16.0,1438.502686,1947.0,129.081419,145.0,441.7,342.507733
00-0036358,DAL,2023,C.Lamb,181,12.352115,12.0,1393.984985,1749.0,117.402901,135.0,381.9,330.914094
00-0031381,LV,2022,D.Adams,180,10.494546,14.0,1526.758789,1516.0,112.967685,100.0,341.6,328.610843
00-0036322,MIN,2022,J.Jefferson,184,10.750579,8.0,1458.707886,1819.0,116.943704,128.0,357.9,327.317971
00-0036900,CIN,2024,J.Chase,175,12.978581,17.0,1284.327271,1708.0,112.442590,127.0,405.8,318.746810
...,...,...,...,...,...,...,...,...,...,...,...,...
00-0035289,JAX,2020,G.Minshew II,1,0.000601,0.0,0.969162,0.0,0.596584,1.0,1.0,0.697108
00-0019596,NE,2018,T.Brady,1,0.002768,0.0,3.223743,6.0,0.329838,1.0,1.6,0.668823
00-0029263,SEA,2018,R.Wilson,1,0.002653,0.0,0.449581,-11.0,0.570121,1.0,-0.1,0.630997


In [15]:
receivers_qual.reset_index().sort_values(['receiver_player_id', 'season'])

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,complete_pass,fantasy_points,compositeXFP
940,00-0021547,LAC,2018,A.Gates,45,3.752923,2.0,308.335419,333.0,27.173962,28.0,79.3,80.525043
466,00-0022127,DAL,2019,J.Witten,83,3.288441,4.0,569.323181,529.0,57.494382,63.0,139.9,134.157347
191,00-0022921,ARI,2018,L.Fitzgerald,112,5.852049,6.0,880.626404,734.0,72.544649,69.0,178.4,195.719583
195,00-0022921,ARI,2019,L.Fitzgerald,109,6.226891,4.0,826.374084,804.0,74.956883,75.0,191.4,194.955640
570,00-0022921,ARI,2020,L.Fitzgerald,72,2.098424,1.0,524.114929,409.0,52.608162,54.0,100.9,117.610205
...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,00-0039901,BUF,2024,K.Coleman,57,5.214764,4.0,496.064392,556.0,29.828633,29.0,108.6,110.723654
1240,00-0039907,NE,2024,J.Polk,33,2.093292,2.0,274.552612,87.0,18.926514,12.0,32.7,58.941528
198,00-0039915,LAC,2024,L.McConkey,112,4.712579,7.0,926.537842,1155.0,73.414770,82.0,239.5,194.344032
914,00-0039916,SF,2024,R.Pearsall,46,1.964631,3.0,415.386536,400.0,29.508494,31.0,89.0,82.834937


In [16]:
df = receivers_qual.reset_index().sort_values(['receiver_player_id', 'season'])

# Create lagged columns for the next season's stats
for col in ['xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass','fantasy_points', 'compositeXFP']:
    df[f'{col}_next'] = df.groupby('receiver_player_id')[col].shift(-1)

# Drop rows where the next season's data is missing (last season for each player)
df_lagged = df.dropna(subset=[f'{col}_next' for col in ['xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass','fantasy_points', 'compositeXFP']])

In [17]:
df.head(20)

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,fantasy_points,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,fantasy_points_next,compositeXFP_next
940,00-0021547,LAC,2018,A.Gates,45,3.752923,2.0,308.335419,333.0,27.173962,...,79.3,80.525043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,00-0022127,DAL,2019,J.Witten,83,3.288441,4.0,569.323181,529.0,57.494382,...,139.9,134.157347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,00-0022921,ARI,2018,L.Fitzgerald,112,5.852049,6.0,880.626404,734.0,72.544649,...,178.4,195.719583,6.226891,4.0,826.374084,804.0,74.956883,75.0,191.4,194.955640
195,00-0022921,ARI,2019,L.Fitzgerald,109,6.226891,4.0,826.374084,804.0,74.956883,...,191.4,194.955640,2.098424,1.0,524.114929,409.0,52.608162,54.0,100.9,117.610205
570,00-0022921,ARI,2020,L.Fitzgerald,72,2.098424,1.0,524.114929,409.0,52.608162,...,100.9,117.610205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,00-0022943,NO,2018,B.Watson,46,4.236354,2.0,367.110840,400.0,29.137302,...,87.0,91.266512,1.276046,0.0,177.596542,173.0,15.453680,17.0,34.3,40.869614
1560,00-0022943,NE,2019,B.Watson,24,1.276046,0.0,177.596542,173.0,15.453680,...,34.3,40.869614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1636,00-0023564,PHI,2018,D.Sproles,23,1.313301,2.0,121.192467,160.0,17.396862,...,43.0,37.395913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1232,00-0024221,WAS,2018,V.Davis,36,0.967655,2.0,311.169250,367.0,22.395950,...,73.7,59.318803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1398,00-0024243,GB,2021,M.Lewis,28,2.228251,0.0,160.090912,214.0,19.881775,...,44.4,49.260372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_lagged

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,fantasy_points,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,fantasy_points_next,compositeXFP_next
191,00-0022921,ARI,2018,L.Fitzgerald,112,5.852049,6.0,880.626404,734.0,72.544649,...,178.4,195.719583,6.226891,4.0,826.374084,804.0,74.956883,75.0,191.4,194.955640
195,00-0022921,ARI,2019,L.Fitzgerald,109,6.226891,4.0,826.374084,804.0,74.956883,...,191.4,194.955640,2.098424,1.0,524.114929,409.0,52.608162,54.0,100.9,117.610205
820,00-0022943,NO,2018,B.Watson,46,4.236354,2.0,367.110840,400.0,29.137302,...,87.0,91.266512,1.276046,0.0,177.596542,173.0,15.453680,17.0,34.3,40.869614
1654,00-0025394,WAS,2018,A.Peterson,26,0.391267,1.0,141.563919,208.0,20.414077,...,46.8,36.918073,0.355608,0.0,106.821838,142.0,17.924322,17.0,31.2,30.740152
1245,00-0025396,NO,2018,T.Ginn,30,2.191082,2.0,278.764801,209.0,17.385297,...,49.9,58.408270,3.995163,2.0,496.046753,421.0,32.226284,30.0,84.1,105.801939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,00-0039067,KC,2023,R.Rice,102,5.365247,7.0,655.570190,938.0,72.821294,...,214.8,170.569795,0.512708,2.0,212.762741,288.0,21.905786,24.0,64.8,46.258308
27,00-0039075,LA,2023,P.Nacua,160,6.651248,6.0,1286.854736,1486.0,108.953619,...,289.6,277.546585,3.597497,3.0,799.447754,990.0,74.036345,79.0,196.0,175.566108
678,00-0039139,DET,2023,J.Gibbs,71,1.722214,1.0,384.768494,316.0,56.332290,...,95.6,105.142424,2.585521,3.0,328.643005,476.0,49.370975,52.0,117.6,97.748403
285,00-0039146,GB,2023,J.Reed,94,5.999262,8.0,733.035645,793.0,59.541225,...,191.3,168.840368,3.335935,6.0,576.351318,857.0,48.792778,55.0,176.7,126.443523


In [19]:
df_lagged.tail(11)

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,fantasy_points,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,fantasy_points_next,compositeXFP_next
1243,00-0039040,MIA,2023,D.Achane,37,1.817977,3.0,201.269119,197.0,27.743733,...,64.7,58.778509,2.899085,6.0,471.492645,569.0,70.264767,78.0,170.9,134.808540
557,00-0039052,TB,2023,T.Palmer,68,3.079318,3.0,568.028015,385.0,43.548505,...,95.5,118.827217,0.868923,1.0,215.105270,172.0,13.136029,12.0,35.2,39.860092
437,00-0039062,CAR,2023,J.Mingo,85,2.194410,0.0,719.073181,418.0,55.025730,...,90.8,140.099508,0.412739,0.0,214.433823,121.0,17.635554,12.0,24.1,41.555372
234,00-0039064,BAL,2023,Z.Flowers,108,4.372850,5.0,834.463684,858.0,72.257856,...,192.8,181.941329,4.782156,4.0,1000.265564,1059.0,74.584208,74.0,203.9,203.303704
163,00-0039065,DET,2023,S.LaPorta,120,5.859093,10.0,860.667725,889.0,83.040305,...,234.9,204.261640,6.238589,7.0,585.452026,726.0,56.613672,60.0,174.6,152.590409
1155,00-0039066,LV,2023,M.Mayer,40,1.186765,2.0,282.026611,304.0,29.314895,...,69.4,64.638143,0.799751,0.0,230.881302,156.0,23.687605,21.0,36.6,51.574243
280,00-0039067,KC,2023,R.Rice,102,5.365247,7.0,655.570190,938.0,72.821294,...,214.8,170.569795,0.512708,2.0,212.762741,288.0,21.905786,24.0,64.8,46.258308
27,00-0039075,LA,2023,P.Nacua,160,6.651248,6.0,1286.854736,1486.0,108.953619,...,289.6,277.546585,3.597497,3.0,799.447754,990.0,74.036345,79.0,196.0,175.566108
678,00-0039139,DET,2023,J.Gibbs,71,1.722214,1.0,384.768494,316.0,56.332290,...,95.6,105.142424,2.585521,3.0,328.643005,476.0,49.370975,52.0,117.6,97.748403
285,00-0039146,GB,2023,J.Reed,94,5.999262,8.0,733.035645,793.0,59.541225,...,191.3,168.840368,3.335935,6.0,576.351318,857.0,48.792778,55.0,176.7,126.443523


In [20]:
correlations = {}

# Loop through each stat to calculate its correlation with the following season
for col in ['xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass', 'fantasy_points', 'compositeXFP']:
    correlation = df_lagged[col].corr(df_lagged[f'{col}_next'])
    #print(df_lagged)
    correlations[col] = correlation

# Display the results
correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation with Next Season'])
print(correlations_df)

                Correlation with Next Season
xTDs                                0.636815
pass_touchdown                      0.481854
xYards                              0.708752
yards_gained                        0.682051
cp                                  0.608767
complete_pass                       0.616703
fantasy_points                      0.654273
compositeXFP                        0.675241


In [21]:
#correlations_df.drop(['xFPs'],inplace=True)

In [22]:
correlations = {}

# Loop through each stat to calculate its correlation with the following season
for metric in ['pass_touchdown', 'yards_gained', 'complete_pass','fantasy_points']:
    if metric == 'pass_touchdown':
        predictor = 'xTDs'
    if metric == 'yards_gained':
        predictor = 'xYards'
    if metric == 'complete_pass':
        predictor = 'cp'
    if metric == 'fantasy_points':
        predictor = 'compositeXFP'
    #print(f"{predictor} to predict {metric}")
    correlation = df_lagged[predictor].corr(df_lagged[f'{metric}_next'])
    correlations[metric] = correlation

# Display the results
x_stat_correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Expected Stat Correlation with Actual Next Season Target'])
print(x_stat_correlations_df)

                Expected Stat Correlation with Actual Next Season Target
pass_touchdown                                           0.502959       
yards_gained                                             0.672379       
complete_pass                                            0.602412       
fantasy_points                                           0.640200       


In [23]:
correlations_df.merge(x_stat_correlations_df,right_index=True,left_index=True,how='outer').sort_values('Correlation with Next Season',ascending=False)

,Correlation with Next Season,Expected Stat Correlation with Actual Next Season Target
xYards,0.708752,NaN
yards_gained,0.682051,0.672379
compositeXFP,0.675241,NaN
fantasy_points,0.654273,0.640200
xTDs,0.636815,NaN
complete_pass,0.616703,0.602412
cp,0.608767,NaN
pass_touchdown,0.481854,0.502959


In [24]:
df_lagged[['xTDs','pass_touchdown','pass_touchdown_next']].corr()

,xTDs,pass_touchdown,pass_touchdown_next
xTDs,1.000000,0.825411,0.502959
pass_touchdown,0.825411,1.000000,0.481854
pass_touchdown_next,0.502959,0.481854,1.000000


In [25]:
df_lagged[['xYards','yards_gained','yards_gained_next']].corr()

,xYards,yards_gained,yards_gained_next
xYards,1.000000,0.961546,0.672379
yards_gained,0.961546,1.000000,0.682051
yards_gained_next,0.672379,0.682051,1.000000


In [26]:
df_lagged

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,fantasy_points,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,fantasy_points_next,compositeXFP_next
191,00-0022921,ARI,2018,L.Fitzgerald,112,5.852049,6.0,880.626404,734.0,72.544649,...,178.4,195.719583,6.226891,4.0,826.374084,804.0,74.956883,75.0,191.4,194.955640
195,00-0022921,ARI,2019,L.Fitzgerald,109,6.226891,4.0,826.374084,804.0,74.956883,...,191.4,194.955640,2.098424,1.0,524.114929,409.0,52.608162,54.0,100.9,117.610205
820,00-0022943,NO,2018,B.Watson,46,4.236354,2.0,367.110840,400.0,29.137302,...,87.0,91.266512,1.276046,0.0,177.596542,173.0,15.453680,17.0,34.3,40.869614
1654,00-0025394,WAS,2018,A.Peterson,26,0.391267,1.0,141.563919,208.0,20.414077,...,46.8,36.918073,0.355608,0.0,106.821838,142.0,17.924322,17.0,31.2,30.740152
1245,00-0025396,NO,2018,T.Ginn,30,2.191082,2.0,278.764801,209.0,17.385297,...,49.9,58.408270,3.995163,2.0,496.046753,421.0,32.226284,30.0,84.1,105.801939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,00-0039067,KC,2023,R.Rice,102,5.365247,7.0,655.570190,938.0,72.821294,...,214.8,170.569795,0.512708,2.0,212.762741,288.0,21.905786,24.0,64.8,46.258308
27,00-0039075,LA,2023,P.Nacua,160,6.651248,6.0,1286.854736,1486.0,108.953619,...,289.6,277.546585,3.597497,3.0,799.447754,990.0,74.036345,79.0,196.0,175.566108
678,00-0039139,DET,2023,J.Gibbs,71,1.722214,1.0,384.768494,316.0,56.332290,...,95.6,105.142424,2.585521,3.0,328.643005,476.0,49.370975,52.0,117.6,97.748403
285,00-0039146,GB,2023,J.Reed,94,5.999262,8.0,733.035645,793.0,59.541225,...,191.3,168.840368,3.335935,6.0,576.351318,857.0,48.792778,55.0,176.7,126.443523


# Intra-season Comparison

In [27]:
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    #throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


In [28]:
new_predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]

test_df = df[new_predictors]

test_df.head()

,air_yards,yardline_100,ydstogo,down,pass_location,qb_hit,end_zone_target,distance_to_EZ_after_target
2,8.0,80.0,15,1.0,right,0.0,False,72.0
5,4.0,39.0,10,1.0,right,0.0,False,35.0
6,-3.0,39.0,10,2.0,left,0.0,False,42.0
7,24.0,39.0,10,3.0,left,0.0,False,15.0
10,1.0,1.0,1,3.0,right,0.0,True,0.0


In [29]:
    
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    #throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


In [30]:
test_df = pd.get_dummies(test_df, columns=['pass_location'], drop_first=True)


# Add predictions to the new dataset (optional)
df['xYards'] = yardage_model.predict(test_df)
df['xTDs'] = touchdown_model.predict_proba(test_df)[:,1]
#df['xFPs'] = best_model.predict(test_df)
df['compositeXFP'] = df['cp'] * ppr + df['xTDs'] * 6 + df['xYards'] * 0.1

C:\Users\src15\AppData\Local\Temp\ipykernel_24468\896432768.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xYards'] = yardage_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_24468\896432768.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xTDs'] = touchdown_model.predict_proba(test_df)[:,1]
C:\Users\src15\AppData\Local\Temp\ipykernel_24468\896432768.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [31]:

# Filter weeks 1-9 and weeks 10-18
first_half = df[df['week'].between(1, 9)]
second_half = df[df['week'].between(10, 18)]

# Define metrics to aggregate
metrics = ['pass','xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass',  'fantasy_points', 'compositeXFP','end_zone_target']

# Aggregate metrics for each player-season for weeks 1-9
first_half_agg = first_half.groupby(['receiver_player_id', 'season'])[metrics].sum()
first_half_agg = first_half_agg.add_suffix('_wk1_9')

# Aggregate metrics for each player-season for weeks 10-18
second_half_agg = second_half.groupby(['receiver_player_id', 'season'])[metrics].sum()
second_half_agg = second_half_agg.add_suffix('_wk10_18')

# Merge the two aggregated DataFrames
df_aggregated = pd.merge(first_half_agg, second_half_agg, left_index=True, right_index=True)


In [32]:
df_aggregated = df_aggregated[(df_aggregated['pass_wk1_9'] >= 5) & (df_aggregated['pass_wk10_18'] >= 5)]


In [33]:
# Create a dictionary to store correlations for each metric
correlations = {}

# Calculate correlation for each metric between weeks 1-9 and weeks 10-18
for metric in metrics:
    correlation = df_aggregated[f'{metric}_wk1_9'].corr(df_aggregated[f'{metric}_wk10_18'])
    correlations[metric] = correlation

# Display the correlation results
correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation between Weeks 1-9 and 10-18'])
print(correlations_df)


                 Correlation between Weeks 1-9 and 10-18
pass                                            0.711110
xTDs                                            0.570537
pass_touchdown                                  0.427380
xYards                                          0.750812
yards_gained                                    0.706406
cp                                              0.686568
complete_pass                                   0.683898
fantasy_points                                  0.685264
compositeXFP                                    0.719506
end_zone_target                                 0.502806


In [34]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9','end_zone_target_wk1_9']].corr()

,fantasy_points_wk1_9,compositeXFP_wk1_9,fantasy_points_wk10_18,pass_wk1_9,end_zone_target_wk1_9
fantasy_points_wk1_9,1.000000,0.954680,0.685264,0.947273,0.675276
compositeXFP_wk1_9,0.954680,1.000000,0.681885,0.990141,0.718748
fantasy_points_wk10_18,0.685264,0.681885,1.000000,0.672112,0.484984
pass_wk1_9,0.947273,0.990141,0.672112,1.000000,0.641261
end_zone_target_wk1_9,0.675276,0.718748,0.484984,0.641261,1.000000


In [35]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9','pass_wk10_18','yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18','xTDs_wk1_9','pass_touchdown_wk1_9','pass_touchdown_wk10_18','end_zone_target_wk1_9']].corr()[['fantasy_points_wk10_18']].sort_values('fantasy_points_wk10_18',ascending=False)

,fantasy_points_wk10_18
fantasy_points_wk10_18,1.000000
yards_gained_wk10_18,0.974119
pass_wk10_18,0.948361
pass_touchdown_wk10_18,0.822624
yards_gained_wk1_9,0.689174
fantasy_points_wk1_9,0.685264
compositeXFP_wk1_9,0.681885
xYards_wk1_9,0.681601
pass_wk1_9,0.672112
xTDs_wk1_9,0.586152


In [36]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9','pass_wk10_18','yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18','xTDs_wk1_9','pass_touchdown_wk1_9','pass_touchdown_wk10_18']].corr()[['yards_gained_wk10_18']].sort_values('yards_gained_wk10_18',ascending=False)

,yards_gained_wk10_18
yards_gained_wk10_18,1.000000
fantasy_points_wk10_18,0.974119
pass_wk10_18,0.932635
pass_touchdown_wk10_18,0.722741
yards_gained_wk1_9,0.706406
xYards_wk1_9,0.701529
compositeXFP_wk1_9,0.692194
fantasy_points_wk1_9,0.690736
pass_wk1_9,0.674954
xTDs_wk1_9,0.605445


In [37]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9','pass_wk10_18','yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18','xTDs_wk1_9','pass_touchdown_wk1_9','pass_touchdown_wk10_18','end_zone_target_wk1_9']].corr()[['pass_touchdown_wk10_18']].sort_values('pass_touchdown_wk10_18',ascending=False)

,pass_touchdown_wk10_18
pass_touchdown_wk10_18,1.000000
fantasy_points_wk10_18,0.822624
yards_gained_wk10_18,0.722741
pass_wk10_18,0.670831
fantasy_points_wk1_9,0.491995
yards_gained_wk1_9,0.491423
xYards_wk1_9,0.483470
compositeXFP_wk1_9,0.479218
pass_wk1_9,0.458281
xTDs_wk1_9,0.455795


In [38]:
df_aggregated[['yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18']].corr()

,yards_gained_wk1_9,xYards_wk1_9,yards_gained_wk10_18
yards_gained_wk1_9,1.000000,0.948712,0.706406
xYards_wk1_9,0.948712,1.000000,0.701529
yards_gained_wk10_18,0.706406,0.701529,1.000000


In [39]:
df_aggregated[['pass_touchdown_wk1_9','xTDs_wk1_9','pass_touchdown_wk10_18']].corr()

,pass_touchdown_wk1_9,xTDs_wk1_9,pass_touchdown_wk10_18
pass_touchdown_wk1_9,1.000000,0.779691,0.427380
xTDs_wk1_9,0.779691,1.000000,0.455795
pass_touchdown_wk10_18,0.427380,0.455795,1.000000


In [40]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9']].corr()

,fantasy_points_wk1_9,compositeXFP_wk1_9,fantasy_points_wk10_18,pass_wk1_9
fantasy_points_wk1_9,1.000000,0.954680,0.685264,0.947273
compositeXFP_wk1_9,0.954680,1.000000,0.681885,0.990141
fantasy_points_wk10_18,0.685264,0.681885,1.000000,0.672112
pass_wk1_9,0.947273,0.990141,0.672112,1.000000
